<a href="https://colab.research.google.com/github/yenlung/Python-AI-Book/blob/main/%E5%86%92%E9%9A%AA_%E5%BC%B7%E5%8C%96%E5%AD%B8%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras.utils import to_categorical

## 抓取data

In [ ]:
pip install finmind

     |████████████████████████████████| 48 kB 4.9 MB/s 
     |████████████████████████████████| 10.9 MB 24.6 MB/s 
     |████████████████████████████████| 297 kB 54.2 MB/s 
     |████████████████████████████████| 58 kB 3.4 MB/s 
     |████████████████████████████████| 6.4 MB 22.4 MB/s 
     |████████████████████████████████| 793 kB 46.6 MB/s 
     |████████████████████████████████| 135 kB 44.8 MB/s 
     |████████████████████████████████| 1.1 MB 20.5 MB/s 
     |████████████████████████████████| 95 kB 3.8 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 144 kB 78.9 MB/s 
     |████████████████████████████████| 271 kB 24.6 MB/s 
     |████████████████████████████████| 96 kB 6.8 MB/s 
     |████████████████████████████████| 224 kB 76.4 MB/s 
     |████████████████████████████████| 132 kB 77.4 MB/s 
     |████████████████████████████████| 380 kB 74.6 MB/s 
     |████████████████████████████████| 130 kB 62.9 MB/s 
  Created wheel for ta:

In [ ]:
# 取得股價
from FinMind.data import DataLoader

In [ ]:
dl = DataLoader()
# 下載台股股價資料
stock_data = dl.taiwan_stock_daily(
    stock_id='2330', start_date='2010-01-01', end_date='2021-12-31')

In [ ]:
stock_data

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
0,2010-01-04,2330,39511138,2557720928,65.0,65.0,64.0,64.9,0.4,8255
1,2010-01-05,2330,38394084,2464115096,65.0,65.1,63.9,64.5,-0.4,9205
2,2010-01-06,2330,52734385,3390698544,64.5,64.9,63.7,64.9,0.4,12597
3,2010-01-07,2330,53294614,3437221996,64.9,65.0,64.2,64.2,-0.7,11195
4,2010-01-08,2330,48047497,3068341466,63.5,64.3,63.5,64.0,-0.2,9804
...,...,...,...,...,...,...,...,...,...,...
2949,2021-12-24,2330,12008673,7276331245,606.0,609.0,604.0,604.0,-2.0,10444
2950,2021-12-27,2330,16771900,10193462167,604.0,610.0,604.0,606.0,2.0,16637
2951,2021-12-28,2330,35156339,21569506434,610.0,615.0,610.0,615.0,9.0,36241
2952,2021-12-29,2330,25604320,15788043027,615.0,619.0,614.0,616.0,1.0,28804


In [ ]:
#選取所需的欄位
stock_data=stock_data[['date','open','max','min','close']]

In [ ]:
#重新命名表格名稱
stock_data.columns=['TIME','OPEN','MAX','MIN','CLOSE']

In [ ]:
stock_data = stock_data.set_index("TIME")
stock_data

,OPEN,MAX,MIN,CLOSE
TIME,,,,
2010-01-04,65.0,65.0,64.0,64.9
2010-01-05,65.0,65.1,63.9,64.5
2010-01-06,64.5,64.9,63.7,64.9
2010-01-07,64.9,65.0,64.2,64.2
2010-01-08,63.5,64.3,63.5,64.0
...,...,...,...,...
2021-12-24,606.0,609.0,604.0,604.0
2021-12-27,604.0,610.0,604.0,606.0
2021-12-28,610.0,615.0,610.0,615.0


In [ ]:
stock_data.to_csv('stock_data.csv')

## 建立Q表設定

In [ ]:
def q_table(table_name, state_data, next_state_data):
  table_name = pd.DataFrame(columns=['state','action','reward','next_state','q'])
  table_name['state'] = state_data
  table_name['next_state'] = next_state_data
  return table_name

## 交易設定

In [ ]:
#actions
# Hold:0, Buy:1, Sell:2
action = [0, 1, 2]

In [ ]:
# 修改交易
def transaction(loop_num1, loop_num2, table_name):

  #交易初始設定
  hold_money=100000
  hold_money_previous=hold_money
  hold_num=0 
  hold_num_previous=0
  buy_price=0

  for i in range(loop_num1,loop_num2): 
    if table_name['action'][i] ==1 and hold_money_previous >0: # Buy
      sell_num=0
      buy_num=0
      buy_num=hold_money/table_name.state[i+1].iloc[-1]['CLOSE'] # 買入股數
      hold_money=0
      hold_num+=buy_num
      #買入價格
      buy_price = table_name.state[i+1].iloc[-1]['CLOSE']
      # (當期總價值-上一期價值)
      table_name.reward[i]=(hold_money+(hold_num * table_name.state[i+1].iloc[-1]['CLOSE']))/(hold_money_previous + hold_num_previous*table_name.state[i].iloc[-1]['CLOSE'])-1
      hold_money_previous=hold_money
      hold_num_previous=hold_num

    elif table_name['action'][i] ==2 and (table_name.state[i].iloc[-1]['CLOSE']-buy_price)/buy_price >= 0.02: # Sell # 漲多少才賣出
      buy_num=0
      sell_num=hold_num
      hold_money+=sell_num * table_name.state[i+1].iloc[-1]['CLOSE'] # 把股數全數賣出
      hold_num=0
      table_name.reward[i]=(hold_money+(hold_num * table_name.state[i+1].iloc[-1]['CLOSE']))/(hold_money_previous + hold_num_previous*table_name.state[i].iloc[-1]['CLOSE'])-1
      hold_money_previous=hold_money
      hold_num_previous=hold_num

    else: # 沒有交易
      table_name['action'][i] =0
      buy_num=0
      sell_num=0
      table_name.reward[i]=(hold_money+(hold_num * table_name.state[i+1].iloc[-1]['CLOSE']))/(hold_money_previous + hold_num_previous*table_name.state[i].iloc[-1]['CLOSE'])-1
      hold_money_previous=hold_money
      hold_num_previous=hold_num

    table_name.q[i] = table_name.reward[i]
  
  return table_name


## Data放入Q表

In [ ]:
state_data=[]
next_state_data=[]

In [ ]:
# 資料處理成20天一筆
for i in range(len(stock_data)):
  state_data.append(stock_data.iloc[i:i+20,:])
  next_state_data.append(stock_data.iloc[i+1:i+21,:])

In [ ]:
# 建立Q表：df1
df1 = q_table('q_table', state_data, next_state_data)
df1

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
1,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
2,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
3,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
4,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
...,...,...,...,...,...
2949,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
2950,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
2951,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
2952,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN


In [ ]:
df1.state[2690]

,OPEN,MAX,MIN,CLOSE
TIME,,,,
2020-12-04,498.5,505.0,497.5,503.0
2020-12-07,512.0,515.0,506.0,514.0
2020-12-08,514.0,525.0,509.0,524.0
2020-12-09,521.0,524.0,520.0,520.0
2020-12-10,511.0,515.0,510.0,512.0
2020-12-11,517.0,517.0,505.0,516.0
2020-12-14,512.0,513.0,508.0,508.0
2020-12-15,507.0,510.0,504.0,504.0
2020-12-16,509.0,515.0,507.0,512.0


## Q表執行交易

In [ ]:
# Q表動作：隨機動作
for i in range(len(df1)):
  df1["action"][i] = np.random.choice(3)

In [ ]:
# Q表：計算reward、q
transaction(0,len(df1)-1,df1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars


,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,0,-0.011706,OPEN MAX MIN CLOSE TIME ...,-0.011706
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2949,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2950,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2951,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2952,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0


In [ ]:
df1

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,0,-0.011706,OPEN MAX MIN CLOSE TIME ...,-0.011706
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2949,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2950,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2951,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2952,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0


## 訓練資料train

In [ ]:
#2020/12/04-2020/12/31
df1['state'][2690]

,OPEN,MAX,MIN,CLOSE
TIME,,,,
2020-12-04,498.5,505.0,497.5,503.0
2020-12-07,512.0,515.0,506.0,514.0
2020-12-08,514.0,525.0,509.0,524.0
2020-12-09,521.0,524.0,520.0,520.0
2020-12-10,511.0,515.0,510.0,512.0
2020-12-11,517.0,517.0,505.0,516.0
2020-12-14,512.0,513.0,508.0,508.0
2020-12-15,507.0,510.0,504.0,504.0
2020-12-16,509.0,515.0,507.0,512.0


In [ ]:
data_train = df1[:2690].copy()
# 1204:2020/12/04-2020/12/31
data_train

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,0,-0.011706,OPEN MAX MIN CLOSE TIME ...,-0.011706
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2685,OPEN MAX MIN CLOSE TIME ...,0,0.001961,OPEN MAX MIN CLOSE TIME ...,0.001961
2686,OPEN MAX MIN CLOSE TIME ...,0,0.007828,OPEN MAX MIN CLOSE TIME ...,0.007828
2687,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2688,OPEN MAX MIN CLOSE TIME ...,0,0.019417,OPEN MAX MIN CLOSE TIME ...,0.019417


In [ ]:
type(df1.state[0])

pandas.core.frame.DataFrame

In [ ]:
# 正規化，當期狀態20天價格/前一期最後一天收盤價格(價格相對)
# 2010.01.04
data_train.state[0] = data_train.state[0]/54.5 #2009年12月31日 開64.40 高64.50 低63.90 收64.50
for i in range(1,len(data_train)):
  data_train.state[i] = data_train.state[i]/df1.state[i-1].CLOSE.iloc[-1]
for i in range(0,len(data_train)):
  data_train.next_state[i] = data_train.next_state[i]/df1.state[i].CLOSE.iloc[-1]

In [ ]:
# data_train：state, next_state拉平
for i in range(len(data_train.state)):
  data_train.state[i] = np.array(data_train.state[i])
  data_train.state[i] = data_train.state[i].flatten()
for i in range(len(data_train.next_state)):
  data_train.next_state[i]=np.array(data_train.next_state[i])
  data_train.next_state[i] = data_train.next_state[i].flatten()

In [ ]:
data_train

,state,action,reward,next_state,q
0,"[1.1926605504587156, 1.1926605504587156, 1.174...",0,0.0,"[1.056910569105691, 1.0585365853658535, 1.0390...",0.0
1,"[1.056910569105691, 1.0585365853658535, 1.0390...",2,0.0,"[1.0785953177257526, 1.085284280936455, 1.0652...",0.0
2,"[1.0785953177257526, 1.085284280936455, 1.0652...",1,0.0,"[1.1, 1.1016949152542372, 1.088135593220339, 1...",0.0
3,"[1.1, 1.1016949152542372, 1.088135593220339, 1...",0,-0.011706,"[1.0618729096989967, 1.0752508361204014, 1.061...",-0.011706
4,"[1.0618729096989967, 1.0752508361204014, 1.061...",0,-0.032149,"[1.0829103214890017, 1.098138747884941, 1.0744...",-0.032149
...,...,...,...,...,...
2685,"[0.9577603143418467, 0.9666011787819253, 0.955...",0,0.001961,"[0.9666666666666667, 0.9676470588235294, 0.942...",0.001961
2686,"[0.9666666666666667, 0.9676470588235294, 0.942...",0,0.007828,"[0.9579256360078278, 0.958904109589041, 0.9461...",0.007828
2687,"[0.9579256360078278, 0.958904109589041, 0.9461...",0,0.0,"[0.9699029126213592, 0.970873786407767, 0.9582...",0.0
2688,"[0.9699029126213592, 0.970873786407767, 0.9582...",0,0.019417,"[0.9699029126213592, 0.9699029126213592, 0.961...",0.019417


In [ ]:
x_train_state = data_train.state
x_train_state 

0       [1.1926605504587156, 1.1926605504587156, 1.174...
1       [1.056910569105691, 1.0585365853658535, 1.0390...
2       [1.0785953177257526, 1.085284280936455, 1.0652...
3       [1.1, 1.1016949152542372, 1.088135593220339, 1...
4       [1.0618729096989967, 1.0752508361204014, 1.061...
                              ...                        
2685    [0.9577603143418467, 0.9666011787819253, 0.955...
2686    [0.9666666666666667, 0.9676470588235294, 0.942...
2687    [0.9579256360078278, 0.958904109589041, 0.9461...
2688    [0.9699029126213592, 0.970873786407767, 0.9582...
2689    [0.9699029126213592, 0.9699029126213592, 0.961...
Name: state, Length: 2690, dtype: object

In [ ]:
# 一個一個array合成一個大array
x_train_state =np.stack(x_train_state)
x_train_state 

array([[1.19266055, 1.19266055, 1.17431193, ..., 1.12844037, 1.08990826,
        1.12844037],
       [1.05691057, 1.05853659, 1.03902439, ..., 0.98699187, 0.96747967,
        0.97235772],
       [1.07859532, 1.08528428, 1.06521739, ..., 1.01003344, 0.98662207,
        0.98662207],
       ...,
       [0.95792564, 0.95890411, 0.94618395, ..., 1.00782779, 0.99608611,
        1.00782779],
       [0.96990291, 0.97087379, 0.95825243, ..., 1.0038835 , 0.9961165 ,
        1.        ],
       [0.96990291, 0.96990291, 0.96116505, ..., 1.01941748, 0.99805825,
        1.01941748]])

In [ ]:
x_train_state.shape

(2690, 80)

In [ ]:
# 丟入模型，Q表動作one-hot encoding
x_train_a = to_categorical(data_train.action)

# 丟入模型，state+動作合在一起
merged_x_train = np.concatenate((x_train_state, x_train_a), axis=1) 

In [ ]:
data_train.groupby('action').count()

,state,reward,next_state,q
action,,,,
0,2523,2523,2523,2523
1,59,59,59,59
2,108,108,108,108


In [ ]:
x_train_a

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
merged_x_train

array([[1.19266055, 1.19266055, 1.17431193, ..., 1.        , 0.        ,
        0.        ],
       [1.05691057, 1.05853659, 1.03902439, ..., 0.        , 0.        ,
        1.        ],
       [1.07859532, 1.08528428, 1.06521739, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.95792564, 0.95890411, 0.94618395, ..., 1.        , 0.        ,
        0.        ],
       [0.96990291, 0.97087379, 0.95825243, ..., 1.        , 0.        ,
        0.        ],
       [0.96990291, 0.96990291, 0.96116505, ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
# y
y_train = data_train.q.values
y_train

array([0.0, 0.0, 0.0, ..., 0.0, 0.01941747572815533, 0.00952380952380949],
      dtype=object)

In [ ]:
# float64 修改成 float32
merged_x_train = np.float32(merged_x_train)
y_train = np.float32(y_train)

## 建立Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.models import load_model 

In [ ]:
#get number of columns in training data
n_cols = merged_x_train.shape[1]
n_cols 

83

In [ ]:
# create model
q_model = Sequential()
q_model.add(Dense(20, input_dim= n_cols, activation='relu'))
q_model.add(Dropout(0.5))
q_model.add(Dense(10, activation='relu'))
q_model.add(Dropout(0.5))
q_model.add(Dense(10, activation='relu'))
q_model.add(BatchNormalization(momentum=0.9, epsilon=0.001))
q_model.add(Dense(1, activation='tanh'))

# Compile model
q_model.compile(loss='mse', optimizer=SGD(lr=0.001)) 

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
  # Fit the model 
  q_model.fit(merged_x_train, y_train, validation_split=0.1, epochs=50, batch_size=16, verbose=1) 

Epoch 1/50
152/152 [==============================] - 3s 5ms/step - loss: 0.2108 - val_loss: 0.0103
Epoch 2/50
152/152 [==============================] - 1s 3ms/step - loss: 0.1398 - val_loss: 0.0227
Epoch 3/50
152/152 [==============================] - 1s 4ms/step - loss: 0.0921 - val_loss: 0.0187
Epoch 4/50
152/152 [==============================] - 0s 3ms/step - loss: 0.0742 - val_loss: 0.0261
Epoch 5/50
152/152 [==============================] - 1s 4ms/step - loss: 0.0591 - val_loss: 0.0218
Epoch 6/50
152/152 [==============================] - 0s 3ms/step - loss: 0.0481 - val_loss: 0.0329
Epoch 7/50
152/152 [==============================] - 1s 4ms/step - loss: 0.0404 - val_loss: 0.0200
Epoch 8/50
152/152 [==============================] - 1s 4ms/step - loss: 0.0368 - val_loss: 0.0121
Epoch 9/50
152/152 [==============================] - 1s 4ms/step - loss: 0.0341 - val_loss: 0.0099
Epoch 10/50
152/152 [==============================] - 1s 5ms/step - loss: 0.0299 - val_loss: 0.0084

In [ ]:
# #load model
# q_model=load_model("q_model_1")
# q_model = tf.keras.models.load_model('drive/MyDrive/q_model_1')

## 計算最大q值

In [ ]:
a1 = np.array([[1.,0.,0.]]*2690) #1205)
a2 = np.array([[0.,1.,0.]]*2690)
a3 = np.array([[0.,0.,1.]]*2690)

In [ ]:
# data_train St+1
next_state_data =data_train.next_state
next_state_data

# 一個一個array合成一個大array
next_state_data =np.stack(next_state_data) 
next_state_data.shape

(2690, 80)

In [ ]:
next_state_a1 = np.concatenate((next_state_data, a1), axis=1)
next_state_a2 = np.concatenate((next_state_data, a2), axis=1)
next_state_a3 = np.concatenate((next_state_data, a3), axis=1)

In [ ]:
next_state_a1 = np.float32(next_state_a1)
next_state_a2 = np.float32(next_state_a2)
next_state_a3 = np.float32(next_state_a3)

In [ ]:
next_state_a1

array([[1.0569105 , 1.0585365 , 1.0390244 , ..., 1.        , 0.        ,
        0.        ],
       [1.0785953 , 1.0852842 , 1.0652174 , ..., 1.        , 0.        ,
        0.        ],
       [1.1       , 1.101695  , 1.0881356 , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.96990293, 0.9708738 , 0.95825243, ..., 1.        , 0.        ,
        0.        ],
       [0.96990293, 0.96990293, 0.9611651 , ..., 1.        , 0.        ,
        0.        ],
       [0.9495238 , 0.96190476, 0.947619  , ..., 1.        , 0.        ,
        0.        ]], dtype=float32)

In [ ]:
next_q_a1 = q_model.predict(next_state_a1)
next_q_a2 = q_model.predict(next_state_a2)
next_q_a3 = q_model.predict(next_state_a3)

In [ ]:
type(next_q_a1)

numpy.ndarray

In [ ]:
q_model_table = pd.DataFrame()
q_model_table["next_q_a1"] = next_q_a1.flatten()
q_model_table['next_q_a2'] = next_q_a2.flatten()
q_model_table['next_q_a3'] = next_q_a3.flatten()

In [ ]:
q_model_table['max']=''
q_model_table['argmax']=''
for i in range(len(q_model_table)):
  e = np.array([q_model_table.next_q_a1[i],q_model_table.next_q_a2[i],q_model_table.next_q_a3[i]])
  q_model_table['max'][i] = max(e)
  q_model_table['argmax'][i] = np.argmax(e)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
q_model_table

,next_q_a1,next_q_a2,next_q_a3,max,argmax
0,-0.008504,0.008190,-0.003699,0.00819,1
1,-0.008570,0.007491,-0.004712,0.007491,1
2,-0.008128,0.007829,-0.003713,0.007829,1
3,-0.008304,0.008285,-0.004456,0.008285,1
4,-0.008496,0.007139,-0.004556,0.007139,1
...,...,...,...,...,...
2685,-0.008470,0.007172,-0.005550,0.007172,1
2686,-0.008976,0.006368,-0.006659,0.006368,1
2687,-0.008670,0.006785,-0.005814,0.006785,1
2688,-0.008316,0.007235,-0.005783,0.007235,1


In [ ]:
q_model_table.groupby('argmax').count()

,next_q_a1,next_q_a2,next_q_a3,max
argmax,,,,
1,2690,2690,2690,2690


## Q表更新q值

In [ ]:
data_train_update = df1[:2690].copy() #2690:2020/12/04-2020/12/31
data_train_update

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,0,-0.011706,OPEN MAX MIN CLOSE TIME ...,-0.011706
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2685,OPEN MAX MIN CLOSE TIME ...,0,0.001961,OPEN MAX MIN CLOSE TIME ...,0.001961
2686,OPEN MAX MIN CLOSE TIME ...,0,0.007828,OPEN MAX MIN CLOSE TIME ...,0.007828
2687,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2688,OPEN MAX MIN CLOSE TIME ...,0,0.019417,OPEN MAX MIN CLOSE TIME ...,0.019417


In [ ]:
discount_rate =0.2
for i in range(len(data_train_update)):
  data_train_update.q[i]=data_train_update.reward[i]+discount_rate*q_model_table['max'][i]

In [ ]:
data_train_update

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.004389
1,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.004335
2,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.004461
3,OPEN MAX MIN CLOSE TIME ...,0,-0.011706,OPEN MAX MIN CLOSE TIME ...,-0.007221
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.027915
...,...,...,...,...,...
2685,OPEN MAX MIN CLOSE TIME ...,0,0.001961,OPEN MAX MIN CLOSE TIME ...,0.006305
2686,OPEN MAX MIN CLOSE TIME ...,0,0.007828,OPEN MAX MIN CLOSE TIME ...,0.01194
2687,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.00416
2688,OPEN MAX MIN CLOSE TIME ...,0,0.019417,OPEN MAX MIN CLOSE TIME ...,0.023651


In [ ]:
action_p = np.random.rand()
epsilon = 0.85
for i in range(len(data_train_update)):
  if i>0:
    if action_p < epsilon:
      # 隨機動作
      data_train_update.action[i] = np.random.choice(3)
    else:
      # 由模型選擇action
      data_train_update.action[i] = q_model_table['argmax'][i]

In [ ]:
# Q表：計算reward、q
transaction(0,len(data_train_update)-1,data_train_update)

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,0,0.013559,OPEN MAX MIN CLOSE TIME ...,0.013559
3,OPEN MAX MIN CLOSE TIME ...,0,-0.011706,OPEN MAX MIN CLOSE TIME ...,-0.011706
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2685,OPEN MAX MIN CLOSE TIME ...,0,0.001961,OPEN MAX MIN CLOSE TIME ...,0.001961
2686,OPEN MAX MIN CLOSE TIME ...,0,0.007828,OPEN MAX MIN CLOSE TIME ...,0.007828
2687,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2688,OPEN MAX MIN CLOSE TIME ...,0,0.019417,OPEN MAX MIN CLOSE TIME ...,0.019417


## 新的訓練資料(再fit Model)
 

In [ ]:
data_train_update

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,0,0.013559,OPEN MAX MIN CLOSE TIME ...,0.013559
3,OPEN MAX MIN CLOSE TIME ...,0,-0.011706,OPEN MAX MIN CLOSE TIME ...,-0.011706
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2685,OPEN MAX MIN CLOSE TIME ...,0,0.001961,OPEN MAX MIN CLOSE TIME ...,0.001961
2686,OPEN MAX MIN CLOSE TIME ...,0,0.007828,OPEN MAX MIN CLOSE TIME ...,0.007828
2687,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2688,OPEN MAX MIN CLOSE TIME ...,0,0.019417,OPEN MAX MIN CLOSE TIME ...,0.019417


In [ ]:
# 丟入模型，Q表動作one-hot encoding
x_train_a = to_categorical(data_train_update.action,3)

# merge，x_train_state
merged_x_train = np.concatenate((x_train_state, x_train_a), axis=1) 

In [ ]:
y_train = data_train_update['q'].values
# 拉平
y_train=y_train.ravel()

In [ ]:
# float64 修改成 float32
merged_x_train = np.float32(merged_x_train)
y_train = np.float32(y_train)

In [ ]:
# Fit the model 
q_model.fit(merged_x_train, y_train, validation_split=0.1, epochs=50, batch_size=30, verbose=1) 

Epoch 1/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 2.9238e-04
Epoch 2/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0016 - val_loss: 2.9131e-04
Epoch 3/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 3.0087e-04
Epoch 4/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 3.3290e-04
Epoch 5/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 3.1986e-04
Epoch 6/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 3.4667e-04
Epoch 7/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 3.2751e-04
Epoch 8/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0016 - val_loss: 3.4364e-04
Epoch 9/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 3.7094e-04
Epoch 10/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0013 - 

## 測試資料test

In [ ]:
# 2690:2020/12/04-2020/12/31
data_test = df1[2690:2934].copy()
data_test

,state,action,reward,next_state,q
2690,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2691,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2692,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2693,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2694,OPEN MAX MIN CLOSE TIME ...,0,0.026549,OPEN MAX MIN CLOSE TIME ...,0.026549
...,...,...,...,...,...
2929,OPEN MAX MIN CLOSE TIME ...,0,-0.0033,OPEN MAX MIN CLOSE TIME ...,-0.0033
2930,OPEN MAX MIN CLOSE TIME ...,0,0.003311,OPEN MAX MIN CLOSE TIME ...,0.003311
2931,OPEN MAX MIN CLOSE TIME ...,0,0.014851,OPEN MAX MIN CLOSE TIME ...,0.014851
2932,OPEN MAX MIN CLOSE TIME ...,0,0.001626,OPEN MAX MIN CLOSE TIME ...,0.001626


In [ ]:
# data_test：state, next_state拉平
for i in range(len(data_test.state)):
  data_test.state[2690+i] = np.array(data_test.state[2690+i])
  data_test.state[2690+i] = data_test.state[2690+i].flatten()
for i in range(len(data_test.next_state)):
  data_test.next_state[2690+i]=np.array(data_test.next_state[2690+i])
  data_test.next_state[2690+i] = data_test.next_state[2690+i].flatten()

In [ ]:
for i in range(len(data_test)):
  data_test.state[2690+i] = data_test.state[2690+i]/df1.state[2690+i-1].CLOSE.iloc[-1]

In [ ]:
# 丟入模型，調整state維度
x_test_state = data_test.state

# 一個一個array合成一個大array
x_test_state =np.stack(x_test_state)
x_test_state 

array([[0.94952381, 0.96190476, 0.94761905, ..., 1.00952381, 0.99809524,
        1.00952381],
       [0.96603774, 0.97169811, 0.95471698, ..., 1.01886792, 0.99622642,
        1.01132075],
       [0.95895522, 0.97947761, 0.94962687, ..., 1.01119403, 0.99813433,
        1.01119403],
       ...,
       [0.9884106 , 0.99337748, 0.97847682, ..., 1.00993377, 1.        ,
        1.00331126],
       [0.98349835, 0.9950495 , 0.98019802, ..., 1.01485149, 1.00660066,
        1.01485149],
       [0.98373984, 1.        , 0.97886179, ..., 1.00650407, 0.99837398,
        1.00162602]])

In [ ]:
a1 = np.array([[1.,0.,0.]]*len(x_test_state))
a2 = np.array([[0.,1.,0.]]*len(x_test_state))
a3 = np.array([[0.,0.,1.]]*len(x_test_state))

state_t_a1 = np.concatenate((x_test_state, a1), axis=1)
state_t_a2 = np.concatenate((x_test_state, a2), axis=1)
state_t_a3 = np.concatenate((x_test_state, a3), axis=1)

In [ ]:
q_a1 = q_model.predict(state_t_a1) 
q_a2 = q_model.predict(state_t_a2)
q_a3 = q_model.predict(state_t_a3)
q_a1 = q_a1.flatten()
q_a2 = q_a2.flatten()
q_a3 = q_a3.flatten()

In [ ]:
# test_data20天為1筆一一丟入模型，讓他選擇最大q值動作
test_table = pd.DataFrame()
test_table["q_a1"] = q_a1 # hold
test_table['q_a2'] = q_a2 # buy
test_table['q_a3'] = q_a3 # sell
test_table['max']=''
test_table['argmax']=''

for i in range(len(test_table)):
  k = np.array([test_table.q_a1[i],test_table.q_a2[i],test_table.q_a3[i]])
  test_table['max'][i] = max(k)
  test_table['argmax'][i] = np.argmax(k)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
test_table

,q_a1,q_a2,q_a3,max,argmax
0,-0.008826,0.006373,-0.006047,0.006373,1
1,-0.008632,0.006656,-0.006003,0.006656,1
2,-0.008337,0.006531,-0.005607,0.006531,1
3,-0.008330,0.007646,-0.005784,0.007646,1
4,-0.008363,0.007257,-0.005857,0.007257,1
...,...,...,...,...,...
239,-0.008723,0.006703,-0.005901,0.006703,1
240,-0.008514,0.006956,-0.006054,0.006956,1
241,-0.008610,0.006500,-0.005984,0.0065,1
242,-0.008431,0.006351,-0.005528,0.006351,1


In [ ]:
q_table_test=pd.DataFrame()
q_table_test['state'] = df1.state[2690:2934].copy()
q_table_test['action']=test_table.argmax.values
q_table_test['reward'] = ''
q_table_test['q']=''

In [ ]:
# 交易
transaction(2690,2690+len(q_table_test)-1,q_table_test)

,state,action,reward,q
2690,OPEN MAX MIN CLOSE TIME ...,1,0.0,0.0
2691,OPEN MAX MIN CLOSE TIME ...,0,0.011194,0.011194
2692,OPEN MAX MIN CLOSE TIME ...,0,0.012915,0.012915
2693,OPEN MAX MIN CLOSE TIME ...,0,0.029144,0.029144
2694,OPEN MAX MIN CLOSE TIME ...,0,0.026549,0.026549
...,...,...,...,...
2929,OPEN MAX MIN CLOSE TIME ...,0,-0.0033,-0.0033
2930,OPEN MAX MIN CLOSE TIME ...,0,0.003311,0.003311
2931,OPEN MAX MIN CLOSE TIME ...,0,0.014851,0.014851
2932,OPEN MAX MIN CLOSE TIME ...,0,0.001626,0.001626


In [ ]:
q_table_test.groupby('action').count()

,state,reward,q
action,,,
0,242,242,242
1,2,2,2


# save output data

In [ ]:
q_table_test.to_csv('1.csv')

# save model

In [ ]:
q_model.save('q_model_1')

INFO:tensorflow:Assets written to: q_model_1/assets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')